# Clock Drawing Segmentation Inference

This notebook implements a comprehensive inference system for the segmentation of clock drawings from the Montreal Cognitive Assessment (MoCA) test. The notebook loads a pre-trained U-Net++ model with SE-ResNet50 backbone to perform multi-class segmentation of clock drawings, identifies their key components, and evaluates segmentation quality using multiple metrics.

### Authors and Contact Information
- **Diego Aldahir Tovar Ledesma** - diego.tovar@udem.edu
- **Jorge Rodrigo Gómez Mayo** - jorger.gomez@udem.edu

**Organization:** Universidad de Monterrey  
**First created:** April 2025

### Project Overview
This inference pipeline processes clock drawings to segment four key components:
- The entire clock face
- Numbers on the clock face
- Clock hands (hour and minute)
- Clock contour/outline

The system evaluates segmentation quality using multiple metrics including standard IoU, relaxed IoU, Dice coefficient, boundary F1 score, Hausdorff distance, precision, and recall. These metrics provide a comprehensive assessment of segmentation performance for each component.

### Technical Implementation
- **Model Loading:** Pre-trained U-Net++ with SE-ResNet50 encoder
- **Image Processing:** Maintains original aspect ratio with proper normalization
- **Multiple Evaluation Metrics:** Standard and relaxed IoU, Dice, boundary F1, Hausdorff distance
- **Output Format:** Transparent PNG masks for each component with evaluation reports

### Usage Instructions
The notebook requires:
1. A trained model weights file (.pth)
2. Input clock drawing images with "_Background" suffix
3. Optional ground truth masks for evaluation

Outputs include:
- Segmented component masks as transparent PNGs
- Visualization of all segmentation results
- Comprehensive evaluation metrics in text format

## Basic Setup and Configuration

This code sets up the environment for clock drawing segmentation inference. It imports necessary libraries for image processing, deep learning, and visualization, and defines a configuration class with parameters for the segmentation model, including device selection for optimized performance on Apple Silicon hardware.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import glob
from torch import nn
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from scipy.spatial.distance import directed_hausdorff
from skimage import measure
from sklearn.metrics import precision_score, recall_score

# Basic configuration
class Config:
    # Device for inference
    DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    
    # Image size for the model
    IMG_SIZE = 256
    
    # Model
    ENCODER = 'se_resnet50'
    ENCODER_WEIGHTS = 'imagenet'
    CLASSES = ['entire', 'numbers', 'hands', 'contour']
    NUM_CLASSES = len(CLASSES)

# Verify device to ensure MPS is available
print(f"Processing device: {Config.DEVICE}")
print(f"MPS available: {torch.backends.mps.is_available()}" if hasattr(torch.backends, 'mps') else "MPS not available")

## Model Architecture Selection

This code defines a function to build the segmentation model with the specified architecture. It supports three architectures (Unet, Unet++, and Linknet) using the segmentation_models_pytorch library, and configures them with the parameters defined in the Config class. The default architecture is Unet++, which should match the architecture used during training.

In [ ]:
def build_model():
    """
    Builds the segmentation model.
    
    Returns:
        nn.Module: The constructed segmentation model with the specified architecture.
    """
    model_type = "Unet++"
    
    if model_type == "Unet":
        model = smp.Unet(
            encoder_name=Config.ENCODER,
            encoder_weights=Config.ENCODER_WEIGHTS,
            classes=Config.NUM_CLASSES,
            activation='sigmoid'
        )
    elif model_type == "Unet++":
        model = smp.UnetPlusPlus(
            encoder_name=Config.ENCODER,
            encoder_weights=Config.ENCODER_WEIGHTS,
            classes=Config.NUM_CLASSES,
            activation='sigmoid'
        )
    elif model_type == "Linknet":
        model = smp.Linknet(
            encoder_name=Config.ENCODER,
            encoder_weights=Config.ENCODER_WEIGHTS,
            classes=Config.NUM_CLASSES,
            activation='sigmoid'
        )
    else:
        raise ValueError(f"Model type not supported: {model_type}")
    
    print(f"Model built: {model_type} with encoder {Config.ENCODER}")
    return model

## Image Segmentation and Evaluation Functions

This code provides comprehensive functions for evaluating segmentation quality with multiple metrics, and implements an image prediction pipeline. The evaluation metrics include IoU (Intersection over Union), relaxed IoU, Dice coefficient, boundary F1 score, Hausdorff distance, precision, and recall. The prediction function loads a trained model, processes input images, generates segmentation masks, saves them with transparency, and calculates performance metrics when ground truth is available.

In [ ]:
def calculate_iou(pred_mask, gt_mask):
    """
    Calculates the Intersection over Union (IoU) between prediction and ground truth masks.
    
    Args:
        pred_mask (numpy.ndarray): Predicted binary mask.
        gt_mask (numpy.ndarray): Ground truth binary mask.
        
    Returns:
        float: IoU score between 0 and 1.
    """
    pred_mask = (pred_mask > 0).astype(np.uint8)
    gt_mask = (gt_mask > 0).astype(np.uint8)
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    if union == 0:
        return 0.0
    return intersection / union


def calculate_relaxed_iou(pred_mask, gt_mask, tolerance=2):
    """
    Calculates a relaxed IoU with dilated ground truth to be more forgiving on boundaries.
    
    Args:
        pred_mask (numpy.ndarray): Predicted binary mask.
        gt_mask (numpy.ndarray): Ground truth binary mask.
        tolerance (int, optional): Dilation size for relaxed boundaries. Defaults to 2.
        
    Returns:
        float: Relaxed IoU score between 0 and 1.
    """
    pred_mask = (pred_mask > 0).astype(np.uint8)
    gt_mask = (gt_mask > 0).astype(np.uint8)
    kernel = np.ones((tolerance, tolerance), np.uint8)
    gt_dilated = cv2.dilate(gt_mask, kernel, iterations=1)
    intersection = np.logical_and(pred_mask, gt_dilated).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    if union == 0:
        return 0.0
    return intersection / union


def calculate_dice(pred_mask, gt_mask):
    """
    Calculates the Dice coefficient (F1 score for segmentation).
    
    Args:
        pred_mask (numpy.ndarray): Predicted binary mask.
        gt_mask (numpy.ndarray): Ground truth binary mask.
        
    Returns:
        float: Dice coefficient between 0 and 1.
    """
    pred_mask = (pred_mask > 0).astype(np.uint8)
    gt_mask = (gt_mask > 0).astype(np.uint8)
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    total = pred_mask.sum() + gt_mask.sum()
    if total == 0:
        return 1.0
    return (2.0 * intersection) / total


def calculate_boundary_f1(pred_mask, gt_mask):
    """
    Calculates the boundary F1 score, focusing on contour accuracy.
    
    Args:
        pred_mask (numpy.ndarray): Predicted binary mask.
        gt_mask (numpy.ndarray): Ground truth binary mask.
        
    Returns:
        float: Boundary F1 score between 0 and 1.
    """
    pred_contours = measure.find_contours(pred_mask, 0.5)
    gt_contours = measure.find_contours(gt_mask, 0.5)
    if not pred_contours or not gt_contours:
        return 0.0
    pred_points = np.vstack(pred_contours)
    gt_points = np.vstack(gt_contours)
    dists_pred_to_gt = np.min(np.linalg.norm(pred_points[:, None] - gt_points[None], axis=2), axis=1)
    dists_gt_to_pred = np.min(np.linalg.norm(gt_points[:, None] - pred_points[None], axis=2), axis=1)
    precision = np.mean(dists_pred_to_gt < 2.0)
    recall = np.mean(dists_gt_to_pred < 2.0)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)


def calculate_hausdorff(pred_mask, gt_mask):
    """
    Calculates the Hausdorff distance between prediction and ground truth masks.
    
    Args:
        pred_mask (numpy.ndarray): Predicted binary mask.
        gt_mask (numpy.ndarray): Ground truth binary mask.
        
    Returns:
        float: Hausdorff distance (lower is better).
    """
    pred_points = np.argwhere(pred_mask > 0)
    gt_points = np.argwhere(gt_mask > 0)
    if len(pred_points) == 0 or len(gt_points) == 0:
        return float('inf')
    hd1 = directed_hausdorff(pred_points, gt_points)[0]
    hd2 = directed_hausdorff(gt_points, pred_points)[0]
    return max(hd1, hd2)


def calculate_precision_recall(pred_mask, gt_mask):
    """
    Calculates precision and recall metrics.
    
    Args:
        pred_mask (numpy.ndarray): Predicted binary mask.
        gt_mask (numpy.ndarray): Ground truth binary mask.
        
    Returns:
        tuple: (precision, recall) scores between 0 and 1.
    """
    pred_flat = pred_mask.flatten()
    gt_flat = gt_mask.flatten()
    precision = precision_score(gt_flat, pred_flat, zero_division=0)
    recall = recall_score(gt_flat, pred_flat, zero_division=0)
    return precision, recall


def predict_image(model_path, image_path, output_dir, gt_dir=None):
    """
    Processes an image through the segmentation model and evaluates results.
    
    This function loads a trained model, generates segmentation masks for an input image,
    saves the results, and calculates evaluation metrics when ground truth is available.
    
    Args:
        model_path (str): Path to the saved model weights.
        image_path (str): Path to the input image.
        output_dir (str): Directory to save prediction outputs.
        gt_dir (str, optional): Directory containing ground truth masks. Defaults to None.
    """
    os.makedirs(output_dir, exist_ok=True)
    print("Loading model...")
    model = build_model()
    model.load_state_dict(torch.load(model_path, map_location=Config.DEVICE))
    model.to(Config.DEVICE)
    model.eval()

    print(f"Processing image: {image_path}")
    image = cv2.imread(image_path)
    if image is None:
        print(f"ERROR: Could not read image: {image_path}")
        return

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transform = A.Compose([
        A.Resize(Config.IMG_SIZE, Config.IMG_SIZE, interpolation=cv2.INTER_NEAREST),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

    transformed = transform(image=image_rgb)
    image_tensor = transformed['image'].unsqueeze(0).to(Config.DEVICE)

    print("Generating predictions...")
    with torch.no_grad():
        pred = model(image_tensor)

    pred_np = pred.squeeze().cpu().numpy()
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    gt_masks = {}
    if gt_dir:
        img_folder = os.path.dirname(image_path)
        img_name = os.path.basename(image_path).split('_Background')[0]

        for class_name in Config.CLASSES:
            gt_path = os.path.join(img_folder, f"{img_name}_{class_name}.*")
            gt_files = glob.glob(gt_path)
            if gt_files:
                gt_mask = cv2.imread(gt_files[0], cv2.IMREAD_UNCHANGED)
                if gt_mask is not None:
                    if gt_mask.shape[-1] == 4:
                        gt_masks[class_name] = gt_mask[:, :, 3] > 127
                    else:
                        gt_gray = cv2.cvtColor(gt_mask, cv2.COLOR_BGR2GRAY)
                        gt_masks[class_name] = gt_gray > 127

    fig, axes = plt.subplots(1, Config.NUM_CLASSES + 1, figsize=(5*(Config.NUM_CLASSES + 1), 5))
    axes[0].imshow(image_rgb)
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    h, w = image.shape[:2]
    iou_file = open(os.path.join(output_dir, f"{base_name}_iou_scores.txt"), 'w')
    iou_file.write("Evaluation metrics by class:\n")
    iou_file.write("-" * 60 + "\n")

    tolerancias = [2, 3, 5]

    for i, class_name in enumerate(Config.CLASSES):
        pred_class = cv2.resize(pred_np[i], (w, h), interpolation=cv2.INTER_NEAREST)
        pred_binary = (pred_class > 0.5).astype(np.uint8) * 255
        axes[i+1].imshow(pred_binary, cmap='gray')
        axes[i+1].set_title(f'Mask: {class_name}')
        axes[i+1].axis('off')

        rgba = np.zeros((h, w, 4), dtype=np.uint8)
        rgba[..., 0:3] = 255
        rgba[..., 3] = pred_binary
        output_path = os.path.join(output_dir, f"{base_name}_{class_name}.png")
        cv2.imwrite(output_path, rgba)
        print(f"Mask saved: {output_path}")

        if class_name in gt_masks:
            pred_mask_bin = pred_class > 0.5
            gt_mask = gt_masks[class_name]

            iou_score = calculate_iou(pred_mask_bin, gt_mask)
            relaxed_iou_scores = {tol: calculate_relaxed_iou(pred_mask_bin, gt_mask, tolerance=tol) for tol in tolerancias}
            dice_score = calculate_dice(pred_mask_bin, gt_mask)
            bf1_score = calculate_boundary_f1(pred_mask_bin, gt_mask)
            hausdorff = calculate_hausdorff(pred_mask_bin, gt_mask)
            precision, recall = calculate_precision_recall(pred_mask_bin, gt_mask)

            iou_file.write(f"{class_name}:\n")
            iou_file.write(f"  - Standard IoU: {iou_score:.4f}\n")
            for tol in tolerancias:
                iou_file.write(f"  - Relaxed IoU (tol={tol}px): {relaxed_iou_scores[tol]:.4f}\n")
            iou_file.write(f"  - Dice coefficient: {dice_score:.4f}\n")
            iou_file.write(f"  - Boundary F1 (tol=2px): {bf1_score:.4f}\n")
            iou_file.write(f"  - Hausdorff distance: {hausdorff:.2f}\n")
            iou_file.write(f"  - Precision: {precision:.4f}\n")
            iou_file.write(f"  - Recall: {recall:.4f}\n\n")

            print(f"{class_name} -> IoU: {iou_score:.4f}, Dice: {dice_score:.4f}, BF1: {bf1_score:.4f}, HD: {hausdorff:.2f}, Prec: {precision:.4f}, Rec: {recall:.4f}")

    iou_file.close()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_all_masks.png"))
    plt.show()

## Model Inference and Evaluation

This code executes the segmentation model on a specific image, evaluates its performance against ground truth masks, and saves the results. It first locates the model weights, identifies the input image with a "_Background" suffix, and then runs the prediction pipeline to generate segmentation masks and calculate various performance metrics.

In [ ]:
# Base paths
model_path = '/Users/diegotovar/Developer/Tesis/UNet/checkpoints/v6/best_model.pth'
img_folder_path = '/Users/diegotovar/Pictures/MoCA/Images Mask Files/IC00627P01E23PD00011_221130_2'
output_dir = '/Users/diegotovar/Developer/Tesis/UNet/results'

# Search for image with "_Background" suffix
input_images = glob.glob(os.path.join(img_folder_path, "*_Background*"))

if not input_images:
    print(f"ERROR: No image with '_Background' suffix found in {img_folder_path}")
else:
    # Take the first image found
    image_path = input_images[0]
    print(f"Input image: {os.path.basename(image_path)}")
    
    # Ground truth directory is the same as the image directory
    gt_dir = img_folder_path
    
    # Run prediction with IoU calculation
    predictions = predict_image(model_path, image_path, output_dir, gt_dir)
    
    print("\nResults saved in:", output_dir)
    print("Metrics file:", os.path.join(output_dir, os.path.basename(image_path).split('.')[0] + "_iou_scores.txt"))